# CRIADOR AUTOMÁTICO DE VÍDEOS CURTOS LEGENDADOS

#### Instalar bibliotecas

In [ ]:
!pip install -r requirements.txt

#### Passo a passo

- Pegar um video longo
- fatiar em videos menores: 15, 30, 60 com  o moviepy
- para cada vídeo menor gerado:
  - excluir videos que não interessa
  - aplicar logotipo
  - aplicar legenda
  - rotacionar 90º se estiver em formato paisagem
  - proceder a postagem normalmente

#### Importar as Bibliotecas Python

In [1]:
import sys, ffmpeg, yt-dlp
from moviepy.editor import *
from PIL.Image import *
from datetime import datetime

print("Bibliotecas importadas com sucesso")
print(f'Versão do python: {sys.version[0:6]}')

Bibliotecas importadas com sucesso
Versão do python: 3.12.4


#### Baixador de vídeos

In [ ]:
url = 'https://www.youtube.com/watch?v=tJugko3iO0o'
import yt_dlp
from yt_dlp.utils import download_range_func

yt_opts = {
    'verbose': True,
    'force_keyframes_at_cuts': True,
    'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',  # Especifica vídeo e áudio em MP4
    'postprocessors': [{  # Inclui um pós-processador para mesclar vídeo e áudio
        'key': 'FFmpegVideoConvertor',
        'preferedformat': 'mp4',
    }],
}

with yt_dlp.YoutubeDL(yt_opts) as ydl:
    ydl.download(url)



### Informações sobre o vídeo

In [ ]:
video = VideoFileClip(input("Caminho do video que será convertido em vídeos curtos: "))
print(
    f'''
Nome: {video.filename}
Duração: {video.duration}
Tamanho: {video.size}
Aspecto: {video.aspect_ratio}
FPS: {video.fps}

''')

#### Fatiador de Vídeos




In [ ]:
# Solicita a entrada do usuário
tamanho = input(
    '''
Escolha o tamanho dos vídeos curtos:
    (1) - 15 segundos
    (2) - 30 segundos
    (3) - 45 segundos
    (4) - 60 segundos
    (5) - 120 segundos
Digite: 
    '''
)

# Dicionário que simula o switch-case para definir o valor de "passo"
switch = {
    '1': 15,
    '2': 30,
    '3': 45,
    '4': 60,
    '5': 120
}

# Define o valor de "passo" baseado na escolha do usuário
passo = switch.get(tamanho, 15)  # 15 segundos por padrão, caso a entrada não seja válida

# Define as variáveis "inicio" e "fim"
inicio = 0
fim = passo


while(fim<= video.duration):
    try:
        shorts = video.subclip(inicio, fim)    
        shorts.resize(width=480).fx(vfx.fadeout, 1).fx(vfx.fadein, 1)

        # Obter a data e hora atual
        now = datetime.now()

        # Formatar a data e hora como uma string
        timestamp = now.strftime("%Y%m%d_%H%M%S")

        filename = f'saida/video_{timestamp}.mp4'

        shorts.write_videofile(filename, fps=30, threads=1)
        inicio += passo
        fim += passo
        if fim > video.duration:
            fim = video.duration
    except Exception as e:
        break

#### Adicionar logotipo

In [ ]:
path_logo = input("Caminho do logotipo: ") # imagem quadrada e em png

path_video = input('caminho do video a aplicar o logotipo: ')
# path_video = filename



shorts = VideoFileClip(path_video).fx(vfx.fadeout, 1).fx(vfx.fadein, 1)


logotipo = ImageClip(path_logo).set_duration(shorts.duration).margin(right=100, top=100, opacity=0).set_pos(("right", "top"))

video_com_logo = CompositeVideoClip([shorts, logotipo], size=shorts.size)

 # Obter a data e hora atual
now = datetime.now()

# Formatar a data e hora como uma string
timestamp = now.strftime("%Y%m%d_%H%M%S")

filename = f'saida/video_{timestamp}.mp4'

video_com_logo.write_videofile(filename, fps=30, threads=1) # renderiza o arquivo



## Gerador de legendas

## Atenção!
Para gerar as legendas use o arquivo legendador.py
---

#### Adicionar a legenda ao video

In [ ]:
video_path = input('Caminho do Vídeo a ser legendado: ')
# video_path = filename
subtitle_path = input('Caminho da legenda para aplicar ao vídeo: ')

# Obter a data e hora atual
now = datetime.now()

# Formatar a data e hora como uma string
timestamp = now.strftime("%Y%m%d_%H%M%S")

output_path = f'saida/video_{timestamp}.mp4'

try:
    # Comando para adicionar legenda ao vídeo e manter o áudio original
    ffmpeg.input(video_path)\
        .output(output_path, vf=f"subtitles={subtitle_path}", acodec="copy")\
        .run(overwrite_output=True)
    print(f"Vídeo legendado gerado com sucesso: {output_path}")
except ffmpeg.Error as e:
    print(f"Ocorreu um erro ao processar o vídeo: {e}")

#### Rotacionador de vídeo

In [ ]:
path_video = input("Caminho do video: ")

# Obter a data e hora atual
now = datetime.now()

# Formatar a data e hora como uma string
timestamp = now.strftime("%Y%m%d_%H%M%S")
    
clip = VideoFileClip(path_video)

if clip.size[0]> clip.size[1]:
        clip = clip.rotate(-90)
        clip.write_videofile(f'saida/video_{timestamp}.mp4', fps=30, threads=1)

# Efeitos adicionais com moviepy

#### Adicionar um gif dançante ou de ação no clip

In [ ]:
path_video = input("Caminho relativo do vídeo: ")
path_gif = input("Caminho relativo do gif: ")

# Carrega o vídeo 'video.mp4' e seleciona um trecho de 0 a 5 segundos
video = VideoFileClip(path_video).subclip(0, 30)


# Carrega o GIF 'meugif.gif', define algumas propriedades e o posiciona no vídeo
reacao = VideoFileClip(
    path_gif, has_mask=True).loop().set_duration(video.duration).margin(left=0, bottom=0, opacity=0).set_pos(("left", "bottom"))

# Cria um vídeo composto com o vídeo principal e a reação, adiciona efeitos de fade in e fade out
output_video = CompositeVideoClip([video, reacao], size=video.size).fx(vfx.fadeout, 1).fx(vfx.fadein, 1)

# Renderiza o vídeo final e o salva como 'output.mp4'
output_video.write_videofile('output.mp4', fps=30, threads=1)


#### Adicionar um áudio a um vídeo

In [ ]:
path_video = input("Caminho relativo do vídeo: ")

# Carrega o vídeo 'video.mp4' e seleciona um trecho de 0 a 5 segundos
video = VideoFileClip(path_video)
# ADICIONAR ÁUDIO AO VÍDEO
path_audio = input("Caminho relativo do audio para aplicar ao vídeo: ")

# Carrega o áudio 'audio.mp3' e define sua duração para a mesma do vídeo
audio = AudioFileClip(path_audio).set_duration(video.duration)

# Define o áudio do vídeo
video = video.set_audio(audio)

# Salva o vídeo
video.write_videofile(f'{path_video.replace(".", "_")}_com_audio.mp4', fps=30, threads=1)

#### panqueca com 3 partes do mesmo video em tempos diferentes
uso de clips_array para deixar um vídeo sobre o outro

In [ ]:
path_video = input("caminho relativo do video: ")

video1 = VideoFileClip(path_video).subclip(0, 15).without_audio()

video2 = VideoFileClip(path_video).subclip(10, 25).without_audio()

video3 = VideoFileClip(path_video).subclip(15, 30).without_audio()

# Cria um arranjo de vídeos em uma "panqueca" (stack) vertical
panqueca_de_videos = clips_array(
    [
        [video1],  # Primeiro vídeo
        [video2],  # Segundo vídeo
        [video3],  # Terceiro vídeo
    ]
)

# Renderiza o vídeo composto e o salva como 'output.mp4'
panqueca_de_videos.write_videofile('output.mp4', fps=24, threads=1)


#### Adicionar clip de texto

In [ ]:
path_video = input('Adicionar o vídeo: ')
video = VideoFileClip(path_video).subclip(1, 10).without_audio().resize(0.3)

texto = input("Texto para inserir: ")
font_size = input("Tamanho da fonte: ")

clip_de_texto = TextClip(texto, font ="Ubuntu", fontsize = int(font_size), color ="white", bg_color="red").set_duration(video.duration)
clip_de_texto = clip_de_texto.set_position("center")


compose = CompositeVideoClip([video, clip_de_texto], size=video.size).fx(vfx.fadeout, 1).fx(vfx.fadein, 1)

compose.preview()

# Renderiza o vídeo final e o salva como 'output.mp4'
#compose.write_videofile(f'{path_video.replace(".", "_")}_com_texto.mp4', fps=30, threads=1)


#### Concatecenar Vídeos
Colocar um vídeo depois do outro

In [ ]:
path_video_1 = input('Adicionar o primeiro vídeo: ')
path_video_2 = input('Adicionar o segundo vídeo: ')


# Carrega o vídeo1, seleciona um trecho de 0 a 9 segundos
video1 = VideoFileClip(path_video_1).subclip(0, 3).resize(0.5)

# Carrega o vídeo 'video.mp4', seleciona um trecho de 20 a 29 segundos
video2 = VideoFileClip(path_video_2).subclip(10, 13).resize(0.5)

# Concatenando os clipes de vídeo 'video1' e 'video2' utilizando o método "compose"
# Adiciona efeitos de fade in e fade out de 3 segundos
novo_video = concatenate_videoclips([video1, video2], method="compose").fx(vfx.fadeout, 3).fx(vfx.fadein, 3)

# Obter a data e hora atual
now = datetime.now()

# Formatar a data e hora como uma string
timestamp = now.strftime("%Y%m%d_%H%M%S")

# Renderiza o novo vídeo concatenado e o salva como 'saida.mp4'
novo_video.write_videofile(f'saida/video_{timestamp}.mp4', fps=24, threads=1)
